# Cogito Evolution - Google Colab

**GPU-accelerated evolutionary simulation with auto-sync to GitHub**

Before running:
1. **Runtime → Change runtime type → T4 GPU**
2. Click Save

## 0. Verify GPU Selection

In [ ]:
import torch

# Check GPU availability
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"Memory: {gpu_memory:.1f} GB")
    print("Status: GPU ENABLED")
else:
    print("WARNING: No GPU detected!")
    print("Go to: Runtime → Change runtime type → T4 GPU")

In [ ]:
# Detailed GPU info
!nvidia-smi

## 1. Setup GitHub Access

**Required**: Create a GitHub Personal Access Token (PAT)
1. Go to: https://github.com/settings/tokens
2. Generate new token (classic)
3. Select `repo` scope
4. Copy the token

**Security**: Use Colab Secrets (key icon on left sidebar)
- Add secret named `GITHUB_TOKEN`
- Paste your token as value

In [ ]:
from google.colab import userdata
import os

# Get GitHub token from Colab secrets
try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    if GITHUB_TOKEN:
        print("GitHub token found!")
    else:
        print("WARNING: GITHUB_TOKEN not set in Colab secrets")
        print("Results will NOT be pushed to GitHub")
except:
    print("WARNING: Could not access Colab secrets")
    print("Results will NOT be pushed to GitHub")
    GITHUB_TOKEN = None

## 2. Clone Repository

In [ ]:
import subprocess

# Clone with token if available
if GITHUB_TOKEN:
    repo_url = f"https://{GITHUB_TOKEN}@github.com/peter941221/Cogito.git"
else:
    repo_url = "https://github.com/peter941221/Cogito.git"

!rm -rf Cogito
!git clone {repo_url}
%cd Cogito

In [ ]:
# Configure git
!git config user.email "colab@evolution.ai"
!git config user.name "Colab Evolution"

## 3. Install Dependencies

In [ ]:
!pip install -q torch numpy matplotlib scikit-learn scipy

## 4. Run Evolution

**Recommended Configurations:**

| Config | Population | Generations | Lifespan | Time (GPU) |
|--------|------------|-------------|----------|------------|
| Quick  | 30         | 10          | 300      | ~5 min     |
| Standard | 50       | 20          | 500      | ~15 min    |
| Deep   | 50         | 50          | 500      | ~30 min    |
| Full   | 100        | 50          | 500      | ~1 hour    |

In [ ]:
# ============================================
# CHOOSE YOUR CONFIGURATION HERE
# ============================================
CONFIG = "standard"  # Options: "quick", "standard", "deep", "full"

configs = {
    "quick":    {"population": 30, "generations": 10, "lifespan": 300},
    "standard": {"population": 50, "generations": 20, "lifespan": 500},
    "deep":     {"population": 50, "generations": 50, "lifespan": 500},
    "full":     {"population": 100, "generations": 50, "lifespan": 500},
}

cfg = configs[CONFIG]
print(f"Running {CONFIG.upper()} configuration:")
print(f"  Population:  {cfg['population']}")
print(f"  Generations: {cfg['generations']}")
print(f"  Lifespan:    {cfg['lifespan']}")
print(f"  Total steps: {cfg['population'] * cfg['generations'] * cfg['lifespan']:,}")

In [ ]:
# Run evolution with selected config
!python run_evolution.py \
    --population {cfg['population']} \
    --generations {cfg['generations']} \
    --lifespan {cfg['lifespan']}

## 5. View Results

In [ ]:
import json
from pathlib import Path
from datetime import datetime

# List generated files
analysis_dir = Path('data/analysis')
checkpoint_dir = Path('data/checkpoints')

print("=" * 50)
print("RESULTS SUMMARY")
print("=" * 50)

if analysis_dir.exists():
    print("\nAnalysis files:")
    for f in sorted(analysis_dir.glob('*')):
        print(f"  {f.name}")

if checkpoint_dir.exists():
    checkpoints = list(checkpoint_dir.glob('*.json'))
    print(f"\nCheckpoints: {len(checkpoints)} files")
    if checkpoints:
        latest = sorted(checkpoints)[-1]
        print(f"  Latest: {latest.name}")

In [ ]:
# Display latest checkpoint summary
import glob

checkpoint_files = sorted(Path('data/checkpoints').glob('evolution_gen_*.json'))
if checkpoint_files:
    latest = checkpoint_files[-1]
    with open(latest) as f:
        data = json.load(f)
    
    print(f"Latest Checkpoint: {latest.name}")
    print(f"Generation: {data.get('generation', 'N/A')}")
    print(f"Population: {len(data.get('population_genomes', []))}")
    
    if 'history' in data:
        h = data['history']
        print(f"\nBest Fitness: {max(h.get('best_fitness', [0])):.1f}")
        print(f"Avg Fitness: {sum(h.get('avg_fitness', [0]))/len(h.get('avg_fitness', [1])):.1f}")

## 6. Push Results to GitHub

This will create a new branch and push all results.

In [ ]:
if GITHUB_TOKEN:
    from datetime import datetime
    
    # Create results branch
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    branch_name = f"evolution-results/{CONFIG}-{timestamp}"
    
    print(f"Creating branch: {branch_name}")
    
    # Create new branch
    !git checkout -b {branch_name}
    
    # Add results
    !git add data/analysis data/checkpoints
    
    # Commit
    commit_msg = f"evolution: {CONFIG} config - {cfg['population']}pop x {cfg['generations']}gen x {cfg['lifespan']}life"
    !git commit -m "{commit_msg}"
    
    # Push
    print(f"\nPushing to GitHub...")
    !git push origin {branch_name}
    
    print(f"\nResults pushed to branch: {branch_name}")
    print(f"View at: https://github.com/peter941221/Cogito/tree/{branch_name}")
else:
    print("Skipping GitHub push - no token configured")
    print("\nTo enable auto-push:")
    print("1. Create token at https://github.com/settings/tokens")
    print("2. Add to Colab secrets as GITHUB_TOKEN")

## 7. Alternative: Download Results Locally

In [ ]:
# Zip and download results
from google.colab import files

!zip -r evolution_results.zip data/analysis data/checkpoints
files.download('evolution_results.zip')